In [1]:
import datetime
import os
import json
import spacy
nlp = spacy.load('en')

In [2]:
# Loading Training Data
training_data = []
training_data_file = 'training_data.json' 
with open(training_data_file) as data_file: 
    training_data = json.load(data_file)
print ("%s sentences in training data" % len(training_data))

12 sentences in training data


In [4]:
words = []
classes = []
documents = []
ignore_words = ['?', '-PRON-']
# loop through each sentence in our training data
for pattern in training_data:
    # tokenize each word in the sentence
    w = nlp(pattern['sentence'])
    # Stemming and removing words
    # add to our words list
    lemmas = [w1.lemma_ for w1 in w if w1.lemma_ not in ignore_words]
    words.extend(lemmas)
    # add to documents in our corpus
    documents.append(([w1.orth_ for w1 in w],lemmas, pattern['class']))    
    # add to our classes list
    if pattern['class'] not in classes:
        classes.append(pattern['class'])

# remove duplicates
words = list(set(words))

# remove duplicates
classes = list(set(classes))

print (len(documents), "documents")
print (len(classes), "classes", classes)
print (len(words), "unique lemmatized words", words)

12 documents
3 classes ['goodbye', 'sandwich', 'greeting']
20 unique stemmed words ['day', 'today', 'soon', 'go', 'good', 'make', 'how', 'lunch', 'sandwich', 'later', 'have', 'to', 'can', 'for', 'see', 'talk', 'a', 'nice', 'what', 'be']


In [5]:
# create our training data
training = []
output = []
# create an empty array for our output
output_empty = [0] * len(classes)

# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of lemmatized words for the pattern
    pattern_words = doc[1]
    # create our bag of words array
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    training.append(bag)
    # output is a '0' for each tag and '1' for current tag
    output_row = list(output_empty)
    output_row[classes.index(doc[2])] = 1
    output.append(output_row)

# sample training/output
i = 0
w = documents[i][1]
print (w)
print (training[i])
print (output[i])

['how', 'be']
[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
[0, 0, 1]


In [6]:
import numpy as np
import time

# compute sigmoid nonlinearity
def sigmoid(x):
    output = 1/(1+np.exp(-x))
    return output

# convert output of sigmoid function to its derivative
def sigmoid_output_to_derivative(output):
    return output*(1-output)
 
def clean_up_sentence(sentence):
    # lemmatize sentence
    return [word.lemma_ for word in nlp(sentence)]

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=False):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)

    return(np.array(bag))

def think(sentence, show_details=False):
    x = bow(sentence, words, show_details)
    if show_details:
        print ("sentence:", sentence, "\n bow:", x)
    # input layer is our bag of words
    l0 = x
    # matrix multiplication of input and hidden layer
    l1 = sigmoid(np.dot(l0, synapse_0))
    # output layer
    l2 = sigmoid(np.dot(l1, synapse_1))
    return l2

In [7]:
def train(X, y, hidden_neurons=10, alpha=1, epochs=50000, dropout=False, dropout_percent=0.5):

    print ("Training with %s neurons, alpha:%s, dropout:%s %s" % (hidden_neurons, str(alpha), dropout, dropout_percent if dropout else '') )
    print ("Input matrix: %sx%s    Output matrix: %sx%s" % (len(X),len(X[0]),1, len(classes)) )
    np.random.seed(1)

    last_mean_error = 1
    # randomly initialize our weights with mean 0
    synapse_0 = 2*np.random.random((len(X[0]), hidden_neurons)) - 1
    synapse_1 = 2*np.random.random((hidden_neurons, len(classes))) - 1

    prev_synapse_0_weight_update = np.zeros_like(synapse_0)
    prev_synapse_1_weight_update = np.zeros_like(synapse_1)

    synapse_0_direction_count = np.zeros_like(synapse_0)
    synapse_1_direction_count = np.zeros_like(synapse_1)
        
    for j in iter(range(epochs+1)):

        # Feed forward through layers 0, 1, and 2
        layer_0 = X
        layer_1 = sigmoid(np.dot(layer_0, synapse_0))
                
        if(dropout):
            layer_1 *= np.random.binomial([np.ones((len(X),hidden_neurons))],1-dropout_percent)[0] * (1.0/(1-dropout_percent))

        layer_2 = sigmoid(np.dot(layer_1, synapse_1))

        # how much did we miss the target value?
        layer_2_error = y - layer_2

        if (j% 10000) == 0 and j > 5000:
            # if this 10k iteration's error is greater than the last iteration, break out
            if np.mean(np.abs(layer_2_error)) < last_mean_error:
                print ("delta after "+str(j)+" iterations:" + str(np.mean(np.abs(layer_2_error))) )
                last_mean_error = np.mean(np.abs(layer_2_error))
            else:
                print ("break:", np.mean(np.abs(layer_2_error)), ">", last_mean_error )
                break
                
        # in what direction is the target value?
        # were we really sure? if so, don't change too much.
        layer_2_delta = layer_2_error * sigmoid_output_to_derivative(layer_2)

        # how much did each l1 value contribute to the l2 error (according to the weights)?
        layer_1_error = layer_2_delta.dot(synapse_1.T)

        # in what direction is the target l1?
        # were we really sure? if so, don't change too much.
        layer_1_delta = layer_1_error * sigmoid_output_to_derivative(layer_1)
        
        synapse_1_weight_update = (layer_1.T.dot(layer_2_delta))
        synapse_0_weight_update = (layer_0.T.dot(layer_1_delta))
        
        if(j > 0):
            synapse_0_direction_count += np.abs(((synapse_0_weight_update > 0)+0) - ((prev_synapse_0_weight_update > 0) + 0))
            synapse_1_direction_count += np.abs(((synapse_1_weight_update > 0)+0) - ((prev_synapse_1_weight_update > 0) + 0))        
        
        synapse_1 += alpha * synapse_1_weight_update
        synapse_0 += alpha * synapse_0_weight_update
        
        prev_synapse_0_weight_update = synapse_0_weight_update
        prev_synapse_1_weight_update = synapse_1_weight_update

    now = datetime.datetime.now()

    # persist synapses
    synapse = {'synapse0': synapse_0.tolist(), 'synapse1': synapse_1.tolist(),
               'datetime': now.strftime("%Y-%m-%d %H:%M"),
               'words': words,
               'classes': classes
              }
    synapse_file = "synapses.json"

    with open(synapse_file, 'w') as outfile:
        json.dump(synapse, outfile, indent=4, sort_keys=True)
    print ("saved synapses to:", synapse_file)

In [8]:
X = np.array(training)
y = np.array(output)

start_time = time.time()

train(X, y, hidden_neurons=20, alpha=0.1, epochs=100000, dropout=False, dropout_percent=0.2)

elapsed_time = time.time() - start_time
print ("processing time:", elapsed_time, "seconds")

Training with 20 neurons, alpha:0.1, dropout:False 
Input matrix: 12x20    Output matrix: 1x3
delta after 10000 iterations:0.00653916778264
delta after 20000 iterations:0.00446672099744
delta after 30000 iterations:0.00358383883652
delta after 40000 iterations:0.00306843136784
delta after 50000 iterations:0.0027215995973
delta after 60000 iterations:0.00246823480161
delta after 70000 iterations:0.00227293407348
delta after 80000 iterations:0.00211656749352
delta after 90000 iterations:0.00198778453136
delta after 100000 iterations:0.00187937370056
saved synapses to: synapses.json
processing time: 5.982884168624878 seconds


In [28]:
# probability threshold
ERROR_THRESHOLD = 0.2
# load our calculated synapse values
synapse_file = 'synapses.json' 
with open(synapse_file) as data_file: 
    synapse = json.load(data_file) 
    synapse_0 = np.asarray(synapse['synapse0']) 
    synapse_1 = np.asarray(synapse['synapse1'])

def classify(sentence, show_details=False):
    results = think(sentence, show_details)

    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD ] 
    # Sorting by keys (When there are more than one classification)
    results.sort(key=lambda x: x[1], reverse=True) 
    return_results =[[classes[r[0]],r[1]] for r in results]
    print ("%s \n classification: %s" % (sentence, return_results))
    
    entities = [entity.ent_type_ for entity in nlp(sentence)]
    print(entities)
    return return_results

In [19]:
classify("sudo make me a sandwich")
classify("how are you today?")
classify("talk to you tomorrow")
classify("who are you?")
classify("make me some lunch")
classify("how was your lunch today?")
print()
classify("good day", show_details=True)

[ 0.00106174  0.99783664  0.00144957]
[[1, 0.99783664189330612]]
[[1, 0.99783664189330612]]
sudo make me a sandwich 
 classification: [['sandwich', 0.99783664189330612]]
[  1.82560193e-04   9.99811117e-03   9.98724170e-01]
[[2, 0.99872416993621449]]
[[2, 0.99872416993621449]]
how are you today? 
 classification: [['greeting', 0.99872416993621449]]
[ 0.97171336  0.00356912  0.00936723]
[[0, 0.97171336010997222]]
[[0, 0.97171336010997222]]
talk to you tomorrow 
 classification: [['goodbye', 0.97171336010997222]]
[ 0.0102899   0.07786401  0.69677156]
[[2, 0.69677156165110499]]
[[2, 0.69677156165110499]]
who are you? 
 classification: [['greeting', 0.69677156165110499]]
[ 0.01974362  0.96997094  0.00418526]
[[1, 0.96997094182786581]]
[[1, 0.96997094182786581]]
make me some lunch 
 classification: [['sandwich', 0.96997094182786581]]
[  1.07705634e-04   8.24569973e-02   9.95089437e-01]
[[2, 0.99508943650841108]]
[[2, 0.99508943650841108]]
how was your lunch today? 
 classification: [['greeti

[['greeting', 0.99666757415286966]]

In [17]:
classify("Hello my friend, how are you today?")

[  1.82560193e-04   9.99811117e-03   9.98724170e-01]
[[2, 0.99872416993621449]]
[[2, 0.99872416993621449]]
Hello my friend, how are you today? 
 classification: [['greeting', 0.99872416993621449]]


[['greeting', 0.99872416993621449]]

In [29]:
classify('what do you mean?')

what do you mean? 
 classification: [['sandwich', 0.89050821839106653]]
['', '', '', '', '']


[['sandwich', 0.89050821839106653]]

In [34]:
classify('I live in Jerusalem')

I live in Jerusalem 
 classification: []
['', '', '', 'GPE']


[]